# 1. preparation

## import packages

In [ ]:
import os, sys, time
import numpy as np
import mapp4py
from mapp4py import dmd

## block the output of all cores except for one

In [ ]:
from mapp4py import mpi
if mpi().rank!=0:
    with open(os.devnull, 'w') as f:
        sys.stdout = f;

# 2. prepare `mapp4py.dmd.atoms` object

## load intial configuraion

In [ ]:
sim=dmd.atoms.import_cfg(5,'configs/Al_DMD.cfg')

## modify degrees of freedom

In [ ]:
def _(id,x_dof,c_dof,alpha_dof,alpha):
    if id>3197:
        x_dof[0]=False;
        x_dof[1]=False;
        x_dof[2]=False;
        c_dof[0]=False;

sim.do(_)

## assign the force field

In [ ]:
sim.ff_eam_setfl("potentials/NiAl.eam.alloy",[3.0],[2.0])

## set the planck and boltzmann constants

In [ ]:
sim.hP = 4.13566766225 * 0.1 * np.sqrt(1.60217656535/1.66053904020)
sim.kB = 8.617330350e-5

## set the temperature

In [ ]:
sim.temp = 300.0

# 3. run initial minimization 

## define minimization

In [ ]:
min_cg = dmd.min_cg();
min_cg.ntally=1000;
min_cg.e_tol=1.0e-10;
min_cg.ls=mapp4py.ls_brent();
min_cg.ntally=0;

## run minimization

In [ ]:
min_cg.run(sim,10000)

# 4. run cehmical integration 

## modify degrees of freedom

In [ ]:
def _(id,alpha_dof,c):
    if id>3197:
        alpha_dof[0]=False;
    if id==0:
        c[0]=0.0;
        
sim.do(_)

## run chemical integration

In [ ]:
chem = dmd.bdf();
chem.a_tol = 1.0e-6;
chem.ntally = 1;
chem.max_ngmres_iters = 20;
chem.max_nnewton_iters = 100;
chem.max_nsteps = 10000;
chem.nreset = 1;
chem.export = dmd.export_cfg('dumps/dump',10,sort=True,extra_vecs=['c_d'])

In [ ]:
sim.step=0
chem.run(sim,10.0**(30));